In [1]:
import re
import tarfile
from pathlib import Path
from time import time
from functools import reduce
from pprint import pprint
from typing import List, Tuple

import numpy as np
import pandas as pd
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, concatenate, Input,Convolution2D, MaxPooling1D, Dense, Dropout, Flatten,LSTM
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, Dropout, Activation
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.python.keras.models import Sequential
from tqdm import tqdm

In [2]:
#import the already preprocessed dataset 
sdg_df  = pd.read_csv('sdg_classification_data.csv')
sdg_df

Unnamed: 0  extracted                                 extracted_abstract  \
0             0       True  paper attempts empirically analyze compatibili...   
1             1       True  aim study determine nature relationship renewa...   
2             2       True  study combines panel data techniques wavelet s...   
3             3       True  environmental issues dependence fossil fuel so...   
4             4       True  environmental issues dependence fossil fuel so...   
5             5       True  blockchain technology ushering nothing short d...   
6             6       True  blockchain technology ushering nothing short d...   
7             7       True  paper estimates impact renewable energy econom...   
8             8       True  paper estimates impact renewable energy econom...   
9             9       True  increasing concerns environmental economic imp...   
10           10       True  increasing concerns environmental economic imp...   
11           11       True  account continuously increasing electricity co...   
12           12       True  account continuously increasing electricity co...   
13           13       True  paper aims analyze interaction renewable energ...   
14           14       True  paper aims analyze interaction renewable energ...   
15           15       True  middle east north africa mena countries recent...   
16           16       True  middle east north africa mena countries recent...   
17           17       True  study investigates dynamic directional informa...   
18           18       True  study investigates dynamic directional informa...   
19           19       True  united nations announced new sustainable devel...   
20           20       True  united nations announced new sustainable devel...   
21           21      False  current study discusses linear nonlinear relat...   
22           22      False  current study discusses linear nonlinear relat...   
23           23       True  renewable energy sources solar panels energy s...   
24           24      False  currently european union eu population live ho...   
25           25       True  republic ghana aims develop utilise renewable ...   
26           26      False  major challenge facing humans st century strik...   
27           27       True  despite increasing cost competitiveness contin...   
28           28       True  paper explains adverse effects due massive inj...   
29           29       True  paper develop quantitative indicator approach ...   
..          ...        ...                                                ...   
869         869       True  study analyzes effects impacts policies could ...   
870         870      False  easy energy access trigger human social econom...   
871         871      False  china proposed clean energy target together cl...   
872         872       True  background billion people cook solid fuels res...   
873         873       True  background access sustained adoption clean hou...   
874         874       True  background kerosene recently considered relati...   
875         875      False  india recently launched national biomass cooks...   
876         876      False  rural household access clean affordable modern...   
877         877      False  air pollution linked many united nations susta...   
878         878      False  understanding scale nature hydrogens potential...   
879         879      False  evidence north india despite widespread global...   
880         880      False  environmental risk factors especially air wate...   
881         881       True  background acute lower respiratory infections ...   
882         882       True  abstractabstract vehicular pollution environme...   
883         883       True  abstract worlds climate gradually altered burn...   
884         884      False  two primary objectives recent energy initiativ...   
885         885       True  abstractthe increasing price fossil fuels coup...   
886         886      False  associat

In [3]:
sdg_df = sdg_df.drop(['Unnamed: 0'],axis = 1)
sdg_df

extracted                                 extracted_abstract  \
0         True  paper attempts empirically analyze compatibili...   
1         True  aim study determine nature relationship renewa...   
2         True  study combines panel data techniques wavelet s...   
3         True  environmental issues dependence fossil fuel so...   
4         True  environmental issues dependence fossil fuel so...   
5         True  blockchain technology ushering nothing short d...   
6         True  blockchain technology ushering nothing short d...   
7         True  paper estimates impact renewable energy econom...   
8         True  paper estimates impact renewable energy econom...   
9         True  increasing concerns environmental economic imp...   
10        True  increasing concerns environmental economic imp...   
11        True  account continuously increasing electricity co...   
12        True  account continuously increasing electricity co...   
13        True  paper aims analyze interaction renewable energ...   
14        True  paper aims analyze interaction renewable energ...   
15        True  middle east north africa mena countries recent...   
16        True  middle east north africa mena countries recent...   
17        True  study investigates dynamic directional informa...   
18        True  study investigates dynamic directional informa...   
19        True  united nations announced new sustainable devel...   
20        True  united nations announced new sustainable devel...   
21       False  current study discusses linear nonlinear relat...   
22       False  current study discusses linear nonlinear relat...   
23        True  renewable energy sources solar panels energy s...   
24       False  currently european union eu population live ho...   
25        True  republic ghana aims develop utilise renewable ...   
26       False  major challenge facing humans st century strik...   
27        True  despite increasing cost competitiveness contin...   
28        True  paper explains adverse effects due massive inj...   
29        True  paper develop quantitative indicator approach ...   
..         ...                                                ...   
869       True  study analyzes effects impacts policies could ...   
870      False  easy energy access trigger human social econom...   
871      False  china proposed clean energy target together cl...   
872       True  background billion people cook solid fuels res...   
873       True  background access sustained adoption clean hou...   
874       True  background kerosene recently considered relati...   
875      False  india recently launched national biomass cooks...   
876      False  rural household access clean affordable modern...   
877      False  air pollution linked many united nations susta...   
878      False  understanding scale nature hydrogens potential...   
879      False  evidence north india despite widespread global...   
880      False  environmental risk factors especially air wate...   
881       True  background acute lower respiratory infections ...   
882       True  abstractabstract vehicular pollution environme...   
883       True  abstract worlds climate gradually altered burn...   
884      False  two primary objectives recent energy initiativ...   
885       True  abstractthe increasing price fossil fuels coup...   
886      False  associated harmful environmental health social...   
887       True  abstract abstract chinese energy system major ...   
888       True  abstract abstract green renewable biodiesel pr...   
889      False  development renewable energy technologies crit...   
890       True  abstract exergy analysis exa gaining relevance...   
891       True  introductionmahatma gandhi said nature enough ...   
892       True  abstract world modernization every country tri...   
893       True  abstract order foster development nations high...   
894      False  sustainable development goal sdg focuses ensur...   
895

In [4]:
X = sdg_df.drop(['sdg','initial_text','extracted', 'extracted_title_tokenized', 'extracted_abstract_tokenized','initial_text_tokenized'], axis=1)
y = sdg_df['sdg']
X

extracted_abstract  \
0    paper attempts empirically analyze compatibili...   
1    aim study determine nature relationship renewa...   
2    study combines panel data techniques wavelet s...   
3    environmental issues dependence fossil fuel so...   
4    environmental issues dependence fossil fuel so...   
5    blockchain technology ushering nothing short d...   
6    blockchain technology ushering nothing short d...   
7    paper estimates impact renewable energy econom...   
8    paper estimates impact renewable energy econom...   
9    increasing concerns environmental economic imp...   
10   increasing concerns environmental economic imp...   
11   account continuously increasing electricity co...   
12   account continuously increasing electricity co...   
13   paper aims analyze interaction renewable energ...   
14   paper aims analyze interaction renewable energ...   
15   middle east north africa mena countries recent...   
16   middle east north africa mena countries recent...   
17   study investigates dynamic directional informa...   
18   study investigates dynamic directional informa...   
19   united nations announced new sustainable devel...   
20   united nations announced new sustainable devel...   
21   current study discusses linear nonlinear relat...   
22   current study discusses linear nonlinear relat...   
23   renewable energy sources solar panels energy s...   
24   currently european union eu population live ho...   
25   republic ghana aims develop utilise renewable ...   
26   major challenge facing humans st century strik...   
27   despite increasing cost competitiveness contin...   
28   paper explains adverse effects due massive inj...   
29   paper develop quantitative indicator approach ...   
..                                                 ...   
869  study analyzes effects impacts policies could ...   
870  easy energy access trigger human social econom...   
871  china proposed clean energy target together cl...   
872  background billion people cook solid fuels res...   
873  background access sustained adoption clean hou...   
874  background kerosene recently considered relati...   
875  india recently launched national biomass cooks...   
876  rural household access clean affordable modern...   
877  air pollution linked many united nations susta...   
878  understanding scale nature hydrogens potential...   
879  evidence north india despite widespread global...   
880  environmental risk factors especially air wate...   
881  background acute lower respiratory infections ...   
882  abstractabstract vehicular pollution environme...   
883  abstract worlds climate gradually altered burn...   
884  two primary objectives recent energy initiativ...   
885  abstractthe increasing price fossil fuels coup...   
886  associated harmful environmental health social...   
887  abstract abstract chinese energy system major ...   
888  abstract abstract green renewable biodiesel pr...   
889  development renewable energy technologies crit...   
890  abstract exergy analysis exa gaining relevance...   
891  introductionmahatma gandhi said nature enough ...   
892  abstract world modernization every country tri...   
893  abstract order foster development nations high...   
894  sustainable development goal sdg focuses ensur...   
895  three processes urbanization industrialization...   
896  abstract bioenergy production biomass sources ...   
897  exponential rise electricity demand become pri...   
898  recent years ten member countries association ...   

                                       extracted_title  
0    trade liberalization policies aligned renewabl...  
1    oil prices macroeconomic factors policies affe...  
2    energy consumption gdp revisited new panel dat...  
3    multiple perspective modeling simulation appro...  
4    multiple perspective modeling simulation appro...  
5    integrating blockchain technology energy secto...  
6    integrating blockchain technology e

In [5]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [33]:
X_title_train = X_train['extracted_title']
X_abstract_train = X_train['extracted_abstract']
X_title_test = X_test['extracted_title']
X_abstract_test = X_test['extracted_abstract']

maxlen = 500
tokenizer = Tokenizer(num_words=50000)
word_index = tokenizer.word_index

tokenizer.fit_on_texts(X_title_train.astype(str))
X_title_train = tokenizer.texts_to_sequences(X_title_train.astype(str))
X_title_train = pad_sequences(X_title_train, padding='post', maxlen=maxlen)

tokenizer.fit_on_texts(X_abstract_train.astype(str))
X_abstract_train = tokenizer.texts_to_sequences(X_abstract_train.astype(str))
X_abstract_train = pad_sequences(X_abstract_train, padding='post', maxlen=maxlen)

tokenizer.fit_on_texts(X_title_test.astype(str))
X_title_test = tokenizer.texts_to_sequences(X_title_test.astype(str))
X_title_test = pad_sequences(X_title_test, padding='post', maxlen=maxlen)

tokenizer.fit_on_texts(X_abstract_test.astype(str))
X_abstract_test = tokenizer.texts_to_sequences(X_abstract_test.astype(str))
X_abstract_test = pad_sequences(X_abstract_test, padding='post', maxlen=maxlen)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
maxlen = 500
#X_title_train = tokenizer.texts_to_sequences(X1_train)
#X1_test = tokenizer.texts_to_sequences(X1_test)

#X1_train = pad_sequences(X1_train, padding='post', maxlen=maxlen)
#X1_test = pad_sequences(X1_test, padding='post', maxlen=maxlen)
X_title_test

array([[1215, 1547, 1760, ...,    0,    0,    0],
       [1735,  690,   18, ...,    0,    0,    0],
       [ 130,  389,   18, ...,    0,    0,    0],
       ...,
       [  21,  118,  770, ...,    0,    0,    0],
       [ 949,   22,    5, ...,    0,    0,    0],
       [1206,  466,  122, ...,    0,    0,    0]])

In [34]:
vocab_size

13201

In [35]:
input_1 = Input(shape=(maxlen,))
input_2 = Input(shape=(maxlen,))

In [36]:
#Model Setup
import keras
from numpy import zeros
from keras.models import Sequential
from keras.layers import SpatialDropout1D 
from tensorflow.python.keras.callbacks import EarlyStopping
from keras.layers.merge import Concatenate

embedding_matrix = zeros((vocab_size, 300))
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(input_1)
LSTM_Layer_1 = LSTM(300,dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dense_layer_1 =Dense(10, activation='relu')(LSTM_Layer_1)

embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(input_2)
LSTM_Layer_2 = LSTM(300,dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dense_layer_2 =Dense(10, activation='relu')(LSTM_Layer_2)

concat_layer = Concatenate()([dense_layer_1, dense_layer_2])
dense_layer_3 = Dense(10, activation='relu')(concat_layer)
output = Dense(6, activation='softmax')(dense_layer_3)
model = Model(inputs=[input_1, input_2], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())
from keras.utils import plot_model
plot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)

Model: "functional_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 500, 300)     3960300     input_7[0][0]                    
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 500, 300)     3960300     input_8[0][0]                    
______________________________________________________________________________________

In [ ]:
history = model.fit(x=[X_abstract_train, X_abstract_train], y=y_train, batch_size=64, epochs=5, verbose=1, validation_data=([X_title_test, X_abstract_test],y_test))

Epoch 1/5
 3/12 [======>.......................] - ETA: 0s - loss: 1.7918 - acc: 0.046 - ETA: 37:38 - loss: 1.7915 - acc: 0.18 - ETA: 1:02:10 - loss: 1.7911 - acc: 0.2031

In [31]:
score = model.evaluate(x=[X_abstract_test, X_title_test], y=y_test, verbose=1)
score

6/6 [==============================] - ETA: 0s - loss: 1.7551 - acc: 0.187 - ETA: 2s - loss: 1.7585 - acc: 0.203 - ETA: 2s - loss: 1.7549 - acc: 0.187 - ETA: 1s - loss: 1.7540 - acc: 0.218 - ETA: 0s - loss: 1.7570 - acc: 0.218 - ETA: 0s - loss: 1.7591 - acc: 0.222 - 7s 1s/step - loss: 1.7591 - acc: 0.2222


[1.7591444253921509, 0.2222222238779068]